# Example of a real time QC check

This notebook provides an example of retrieving and checking some data from an Argo profile.

In [ ]:
%%capture
# prevent logging output for the download
# download some Argo data
!curl -O https://data-argo.ifremer.fr/dac/aoml/5906305/profiles/R5906305_001.nc

In [ ]:
# import some useful libraries
from argortqcpy.profile import Profile
from argortqcpy.checks import ArgoQcFlag, PressureIncreasingCheck
from netCDF4 import Dataset
import plotly.express as px
import numpy as np

In [ ]:
dataset = Dataset("R5906305_001.nc", "r+")

# modify the pressure data to introduce some non-increasing data
bad_pressure = dataset["PRES"][:]
bad_pressure[0][0:30:5] -= 50
bad_pressure[0][40:100:5] -= 100
bad_pressure[0][105:280:20] -= 50
bad_pressure[0][300:320] = 110
bad_pressure[0][330:400] = -50
bad_pressure[0][480:490] = 1100
bad_pressure[0][495:498] = 5
dataset["PRES"][:] = bad_pressure

# store pressure and temperature for later use after closing the dataset
pressure = dataset["PRES"][:]
temperature = dataset["TEMP"][:]

# set up the profile and run the check
profile = Profile(dataset=dataset)
check = PressureIncreasingCheck(profile=profile, profile_previous=None)
output = check.run()

# extract the bad data indices from the output
bad_indices = output.get_output_flags_for_property("PRES") == ArgoQcFlag.BAD.value

# close the netCDF dataset
dataset.close()

In [ ]:
fig = px.scatter(
    x=temperature[0].data,
    y=pressure[0].data,
    color=bad_indices[0].data,
    labels={
        "x": "Temperature (°C)",
        "y": "Pressure (dbar)",
    },
)
name_map = {"True": "Bad", "False": "Good"}
fig.for_each_trace(lambda trace: trace.update(name=name_map[trace.name]))
fig.layout.yaxis.autorange = "reversed"
fig.show()